In [3]:
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import pickle

In [46]:
# topix銘柄を東証のファイルから取得
df_company = pd.read_excel('data/data_j.xls', sheet_name=0, index_col=0)
df_company = df_company.reset_index()
df_company = df_company.rename(columns={'コード':'code', '銘柄名':'name', '市場・商品区分':'market', '33業種区分':'cat33', '17業種区分':'cat17', '規模区分':'scale'})
df_company = df_company[['code','name', 'market', 'cat33', 'cat17', 'scale']]
df_company = df_company[df_company['scale'].str.contains('TOPIX')]
df_company = df_company[~df_company['scale'].str.contains('TOPIX Small ')]

In [53]:
# category dict
codes = df_company.code.to_numpy()
cat33s = df_company.cat33.to_numpy()
cat17s = df_company.cat17.to_numpy()
cat_dict = {}

for i in range(len(codes)):
    cat_dict[codes[i]] = [cat33s[i], cat17s[i]]

In [15]:
# get date
with open('data/closes_dict.pickle', 'rb') as f:
    closes_dict = pickle.load(f)

dates = []
for i in range(0, 23):
    for j in range(1, 12):
        date_20 = '20{:02}-{:02}-20'.format(i, j)
        dates.append(date_20)
    
closes_dict['date'] = dates

In [45]:
# get close df
df_closes = pd.DataFrame(closes_dict)

In [96]:
# macroの特徴量作成
df_macro = df_closes[df_closes.date <= '2022-06-20'][['date', '^N225', 'USDJPY=X', 'CL=F', '^TNX']]
for i in [3, 6, 12, 24, 36, 48]:
    df_macro['^N225_rel_{}'.format(i)] = df_macro['^N225'] / df_macro['^N225'].transform(lambda x: x.rolling(i, 1).mean())
    df_macro['USDJPY=X_rel_{}'.format(i)] = df_macro['USDJPY=X'] / df_macro['USDJPY=X'].transform(lambda x: x.rolling(i, 1).mean())
    df_macro['CL=F_rel_{}'.format(i)] = df_macro['CL=F'] / df_macro['CL=F'].transform(lambda x: x.rolling(i, 1).mean())
    df_macro['^TNX_rel_{}'.format(i)] = df_macro['^TNX'] / df_macro['^TNX'].transform(lambda x: x.rolling(i, 1).mean())

In [119]:
target_codes = [1332, 1333, 1414, 1417, 1605, 1721, 1801, 1802, 1803, 1808, 1812, 1820, 1860, 1878, 1893, 1911, 1925, 1928, 1944, 1951, 1959, 1963, 2002, 2127, 2175, 2181, 2201, 2206, 2212, 2229, 2264, 2267, 2269, 2270, 2282, 2331, 2371, 2412, 2413, 2427, 2432, 2433, 2492, 2501, 2502, 2503, 2531, 2579, 2587, 2593, 2607, 2651, 2670, 2768, 2784, 2801, 2802, 2809, 2810, 2811, 2815, 2871, 2875, 2897, 2914, 3003, 3038, 3064, 3086, 3088, 3092, 3099, 3101, 3105, 3107, 3116, 3141, 3197, 3231, 3288, 3289, 3291, 3349, 3360, 3382, 3391, 3401, 3402, 3405, 3407, 3436, 3549, 3563, 3591, 3626, 3635, 3659, 3697, 3765, 3769, 3774, 3861, 3863, 3880, 3923, 3941, 3994, 4004, 4005, 4021, 4042, 4043, 4045, 4061, 4062, 4063, 4088, 4091, 4114, 4118, 4151, 4182, 4183, 4185, 4186, 4188, 4202, 4203, 4204, 4205, 4206, 4208, 4272, 4307, 4324, 4401, 4403, 4443, 4452, 4502, 4503, 4506, 4507, 4516, 4519, 4521, 4523, 4527, 4528, 4530, 4534, 4536, 4540, 4543, 4544, 4552, 4568, 4578, 4581, 4587, 4612, 4613, 4631, 4661, 4665, 4666, 4676, 4681, 4684, 4686, 4689, 4704, 4716, 4732, 4739, 4751, 4755, 4768, 4887, 4901, 4902, 4911, 4912, 4919, 4921, 4922, 4927, 4967, 5019, 5020, 5021, 5076, 5101, 5105, 5108, 5110, 5201, 5214, 5232, 5233, 5301, 5332, 5333, 5334, 5401, 5406, 5411, 5444, 5463, 5471, 5486, 5631, 5703, 5706, 5711, 5713, 5714, 5801, 5802, 5901, 5929, 5938, 5947, 5991, 6005, 6028, 6098, 6103, 6113, 6134, 6136, 6141, 6146, 6178, 6201, 6268, 6273, 6301, 6302, 6305, 6326, 6361, 6367, 6370, 6383, 6395, 6406, 6417, 6436, 6448, 6457, 6460, 6465, 6471, 6472, 6473, 6479, 6481, 6501, 6502, 6503, 6504, 6506, 6532, 6586, 6592, 6594, 6645, 6674, 6701, 6702, 6723, 6724, 6728, 6752, 6753, 6754, 6755, 6758, 6762, 6770, 6806, 6841, 6845, 6849, 6856, 6857, 6861, 6869, 6902, 6920, 6923, 6925, 6951, 6952, 6954, 6963, 6965, 6967, 6971, 6976, 6981, 6988, 7011, 7012, 7013, 7164, 7167, 7180, 7181, 7182, 7186, 7201, 7202, 7203, 7205, 7211, 7240, 7259, 7261, 7267, 7269, 7270, 7272, 7276, 7282, 7309, 7313, 7337, 7453, 7458, 7459, 7476, 7518, 7532, 7550, 7616, 7649, 7701, 7730, 7731, 7732, 7733, 7735, 7741, 7747, 7751, 7752, 7780, 7832, 7846, 7911, 7912, 7936, 7947, 7951, 7956, 7966, 7974, 7984, 7988, 8001, 8002, 8012, 8015, 8031, 8035, 8053, 8056, 8058, 8060, 8086, 8088, 8111, 8113, 8129, 8136, 8227, 8233, 8252, 8253, 8267, 8273, 8279, 8282, 8283, 8303, 8304, 8306, 8308, 8309, 8316, 8331, 8334, 8354, 8355, 8359, 8369, 8377, 8382, 8385, 8410, 8411, 8418, 8439, 8473, 8570, 8572, 8591, 8593, 8595, 8601, 8604, 8630, 8697, 8725, 8750, 8766, 8795, 8801, 8802, 8804, 8830, 8876, 8905, 9001, 9003, 9005, 9006, 9007, 9008, 9009, 9020, 9021, 9022, 9024, 9031, 9041, 9042, 9044, 9045, 9048, 9064, 9065, 9072, 9076, 9086, 9101, 9104, 9107, 9142, 9143, 9147, 9201, 9202, 9301, 9364, 9401, 9404, 9409, 9432, 9433, 9434, 9435, 9449, 9468, 9501, 9502, 9503, 9504, 9505, 9506, 9507, 9508, 9509, 9513, 9531, 9532, 9533, 9601, 9602, 9613, 9627, 9684, 9697, 9706, 9719, 9735, 9744, 9766, 9783, 9831, 9832, 9843, 9861, 9962, 9983, 9984, 9987, 9989]

In [128]:
df_all = pd.DataFrame()
for code in target_codes:
    # 個別株のデータの取り出し
    df_i = df_closes[df_closes.date <= '2022-06-20'][['date', code, '^N225']]

    # カテゴリ
    df_i[['cat33', 'cat17']] = cat_dict[1332]

    # 株価の推移
    df_i['stock_pri'] = df_i[code]
    for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
        df_i['stock_pri_{}'.format(i)] = df_i['stock_pri'].shift(i) / df_i['stock_pri']

    # 日経平均に対する値動き
    df_i['rel_pri'] = df_i['stock_pri'] / df_i['^N225']
    for i in range(0, 12):
        df_i['rel_ret_{}'.format(i)] = df_i['rel_pri'].shift(i) / df_i['rel_pri'].shift(i+1)
        df_i['rel_ret_{}'.format(i)] = df_i['rel_pri'].shift(i) / df_i['rel_pri'].shift(i+1)

    # センチメント
    for i in [3, 6, 12, 24, 36, 48]:
        df_i['stock_pri_rel_{}'.format(i)] = df_i['stock_pri'] / df_i['stock_pri'].transform(lambda x: x.rolling(i, 1).mean())
        df_i['rel_pri_rel_{}'.format(i)] = df_i['rel_pri'] / df_i['rel_pri'].transform(lambda x: x.rolling(i, 1).mean())


    df_i['target'] = df_i['rel_pri'].shift(-3)/df_i['rel_pri']
    df_i = df_i.drop(code, axis=1)
    df_i = df_i.drop('^N225', axis=1)
    df_i['code_num'] = code

    df_i = pd.merge(df_i, df_macro, on='date')
    df_i = df_i[df_i.date >= '2010']

    df_all = pd.concat([df_all, df_i])

In [129]:
df_all

,date,cat33,cat17,stock_pri,stock_pri_1,stock_pri_2,stock_pri_3,stock_pri_4,stock_pri_5,stock_pri_6,...,CL=F_rel_24,^TNX_rel_24,^N225_rel_36,USDJPY=X_rel_36,CL=F_rel_36,^TNX_rel_36,^N225_rel_48,USDJPY=X_rel_48,CL=F_rel_48,^TNX_rel_48
110,2010-01-20,水産・農林業,食品,273.0,0.934066,1.000000,1.000000,1.029304,0.908425,0.948718,...,0.935162,1.049125,0.826519,0.863901,1.001703,0.938573,0.791343,0.843626,1.041961,0.891764
111,2010-02-20,水産・農林業,食品,255.0,1.070588,1.000000,1.070588,1.070588,1.101961,0.972549,...,0.970423,1.087928,0.790293,0.873445,1.021546,0.977100,0.749577,0.850191,1.067306,0.923904
112,2010-03-20,水産・農林業,食品,271.0,0.940959,1.007380,0.940959,1.007380,1.007380,1.036900,...,0.985941,1.060105,0.854104,0.870059,1.024710,0.958804,0.804427,0.844866,1.073066,0.904249
113,2010-04-20,水産・農林業,食品,278.0,0.974820,0.917266,0.982014,0.917266,0.982014,0.982014,...,1.028850,1.095503,0.872353,0.901190,1.048257,0.996457,0.814768,0.872937,1.101571,0.936524
114,2010-05-20,水産・農林業,食品,279.0,0.996416,0.971326,0.913978,0.978495,0.913978,0.978495,...,0.853321,0.940465,0.817064,0.878575,0.851355,0.863676,0.756386,0.848348,0.897842,0.807912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2022-02-20,水産・農林業,食品,3145.0,0.923688,1.000000,1.116057,1.165342,1.187599,1.143084,...,1.698975,1.576392,1.111656,1.053332,1.659884,1.236238,1.134113,1.050222,1.587236,1.026261
244,2022-03-20,水産・農林業,食品,3050.0,1.031148,0.952459,1.031148,1.150820,1.201639,1.224590,...,1.885595,1.734239,1.093053,1.084858,1.860029,1.396796,1.116569,1.082123,1.790190,1.143948
245,2022-04-20,水産・農林業,食品,2919.0,1.044878,1.077424,0.995204,1.077424,1.202467,1.255567,...,1.784887,2.195805,1.100798,1.166492,1.782333,1.844921,1.127971,1.165589,1.728125,1.504669
246,2022-05-20,水産・農林業,食品,2727.0,1.070407,1.118445,1.153282,1.065273,1.153282,1.287129,...,1.845640,2.033688,1.075022,1.159776,1.912238,1.806157,1.105331,1.160242,1.871686,1.474148


In [123]:
df_all

""


In [112]:
df_i.columns

Index(['date', '^N225', 'cat33', 'cat17', 'stock_pri', 'stock_pri_1',
       'stock_pri_2', 'stock_pri_3', 'stock_pri_4', 'stock_pri_5',
       'stock_pri_6', 'stock_pri_7', 'stock_pri_8', 'stock_pri_9',
       'stock_pri_10', 'stock_pri_11', 'stock_pri_12', 'rel_pri', 'rel_ret_0',
       'rel_ret_1', 'rel_ret_2', 'rel_ret_3', 'rel_ret_4', 'rel_ret_5',
       'rel_ret_6', 'rel_ret_7', 'rel_ret_8', 'rel_ret_9', 'rel_ret_10',
       'rel_ret_11', 'stock_pri_rel_3', 'rel_pri_rel_3', 'stock_pri_rel_6',
       'rel_pri_rel_6', 'stock_pri_rel_12', 'rel_pri_rel_12',
       'stock_pri_rel_24', 'rel_pri_rel_24', 'stock_pri_rel_36',
       'rel_pri_rel_36', 'stock_pri_rel_48', 'rel_pri_rel_48', 'target'],
      dtype='object')